Goals for this code
1. Take a look at the fields in the Ford and Lyft data
2. Determine what fields are useful to me
3. Determine how to combine them, since both datasets have some different field and field names

BONUS: Make sure my S3 AWS bucket can be accessed by anyone and provide instructions on access credentials so they can easily download it

BONUS 2: Complete in google collabs so easier for anyone to pickup and run code

In [2]:
#I will utilize some code from my past web scraping project and tweak it for this project.
#I will use selenium since that is what I have used in the past and is what I am familiar with

#This currently looks up certain fields for you and enters them into the website. 
#Next would be to automate this to run through all 4 designations I want. Do that at end. Also make cleaner later.
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import math
import regex as re 
from selenium.webdriver.support.ui import Select

from zipfile import ZipFile
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from decimal import Decimal
import numpy as np
import pandas as pd
import math
import os
import io

from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import r2_score
import boto3 

In [7]:
#pull data from S3 and store in aws_data folder
#we already have a copy locally but for other users who don't this code will be helpful


import boto3

# Create an S3 access object
s3 = boto3.client("s3")

s3.download_file(
    Bucket="baywheels.data", Key="ford_data.csv", Filename="aws_data/ford_data_download.csv"
)

s3.download_file(
    Bucket="baywheels.data", Key="lyft_data.csv", Filename="aws_data/lyft_data_download.csv"
)

In [37]:
#Let us start by taking a look at the ford data
df_ford = pd.read_csv('aws_data/ford_data_download.csv')
df_lyft = pd.read_csv('aws_data/lyft_data_download.csv')

C:\Users\gadde\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\gadde\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,8,13,14,15,16,17,18,19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
#Looking at the data types there are a lot of objects that we should change into actual data types
print('ford dataset data types: ')
print(df_ford.dtypes)
print('\nlyft dataset data types: ')
print(df_lyft.dtypes)


ford dataset data types: 
Unnamed: 0                   int64
duration_sec                 int64
start_time                  object
end_time                    object
start_station_id           float64
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             float64
end_station_name            object
end_station_latitude       float64
end_station_longitude      float64
bike_id                      int64
user_type                   object
bike_share_for_all_trip     object
dtype: object

lyft dataset data types: 
Unnamed: 0                   int64
duration_sec               float64
start_time                  object
end_time                    object
start_station_id            object
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id              object
end_station_name            object
end_station_latitude       float64
end_station_longitude  

I have decided to include the following fields in my first-run analysis
1. duration_sec
2. start_time /  started_at                
3. end_time / ended_at
4. user_type / member_casual
5. bike_share_for_all_trip

Reasons for choice
1. Want fields present in both datasets to have longest running data possible
2. Want to limit analysis really to duration and time of day - which is why station info was excluded

Note we may choose to include more fields in a later analysis

In [71]:
#select only columns of interest
col_ford = ['start_time','end_time','user_type','bike_share_for_all_trip']
df_ford_trim = pd.DataFrame()
df_ford_trim = df_ford[col_ford]

col_lyft = ['start_time','started_at','end_time','ended_at','user_type', 'member_casual','bike_share_for_all_trip']
df_lyft_trim = pd.DataFrame()
df_lyft_trim = df_lyft[col_lyft]

In [72]:
#Combine the time columns for the lyft data. Let us use fillna
df_lyft_trim['start_time'] = df_lyft_trim['start_time'].fillna(df_lyft_trim['started_at'])
df_lyft_trim['end_time'] = df_lyft_trim['end_time'].fillna(df_lyft_trim['ended_at'])
df_lyft_trim['user_type'] = df_lyft_trim['user_type'].fillna(df_lyft_trim['member_casual'])

#now drop the alternate named columns since combined
df_lyft_trim = df_lyft_trim.drop(['started_at', 'ended_at', 'member_casual'], axis=1)

C:\Users\gadde\AppData\Local\Temp/ipykernel_6880/782311638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lyft_trim['start_time'] = df_lyft_trim['start_time'].fillna(df_lyft_trim['started_at'])
C:\Users\gadde\AppData\Local\Temp/ipykernel_6880/782311638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lyft_trim['end_time'] = df_lyft_trim['end_time'].fillna(df_lyft_trim['ended_at'])
C:\Users\gadde\AppData\Local\Temp/ipykernel_6880/782311638.py:4: SettingWithCopyWarning: 
A value is trying to be 

In [73]:
df_ford_trim.tail()

,start_time,end_time,user_type,bike_share_for_all_trip
3254320,2019-04-01 00:09:17.5660,2019-04-01 00:12:22.5170,Subscriber,No
3254321,2019-04-01 00:03:02.5730,2019-04-01 00:12:02.0670,Subscriber,Yes
3254322,2019-04-01 00:06:04.2370,2019-04-01 00:10:56.9850,Subscriber,Yes
3254323,2019-04-01 00:01:38.4110,2019-04-01 00:09:29.9650,Subscriber,Yes
3254324,2019-04-01 00:00:28.7290,2019-04-01 00:06:25.0650,Subscriber,No


In [74]:
#Change column data types from object to more reasonable values
df_ford_trim['start_time'] = pd.to_datetime(df_ford_trim['start_time'])
df_ford_trim['end_time'] = pd.to_datetime(df_ford_trim['end_time'])
df_lyft_trim['start_time'] = pd.to_datetime(df_lyft_trim['start_time'])
df_lyft_trim['end_time'] = pd.to_datetime(df_lyft_trim['end_time'])

C:\Users\gadde\AppData\Local\Temp/ipykernel_6880/2406727157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ford_trim['start_time'] = pd.to_datetime(df_ford_trim['start_time'])
C:\Users\gadde\AppData\Local\Temp/ipykernel_6880/2406727157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ford_trim['end_time'] = pd.to_datetime(df_ford_trim['end_time'])


In [75]:
#union the 2 df
df = pd.concat([df_ford_trim, df_lyft_trim])

In [80]:
(df['start_time'] - df['end_time'])

0         -1 days +01:44:49.409000
1         -1 days +02:06:39.225000
2         -1 days +11:17:11.528000
3         -1 days +06:43:47.105000
4         -1 days +11:53:16.430000
                    ...           
6564180          -1 days +23:24:19
6564181          -1 days +23:55:47
6564182          -1 days +23:46:37
6564183          -1 days +23:58:31
6564184          -1 days +23:36:00
Length: 9818510, dtype: timedelta64[ns]